# ⚡ Advanced Performance Optimization

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Advanced<br>
<b>Estimated Time:</b> 25 minutes<br>
<b>Prerequisites:</b> All basic notebooks<br>
<b>Dataset:</b> Large synthetic dataset
</div>

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- ✅ Optimize DeepBridge for large-scale datasets
- ✅ Use parallel processing for faster validation
- ✅ Leverage pre-computed probabilities
- ✅ Optimize memory usage
- ✅ Profile and benchmark your experiments
- ✅ Apply production-grade optimization techniques

---

## 📚 Table of Contents

1. [Introduction](#intro)
2. [Setup](#setup)
3. [Baseline Performance](#baseline)
4. [Optimization 1: Pre-computed Probabilities](#precomputed)
5. [Optimization 2: Parallel Processing](#parallel)
6. [Optimization 3: Memory Management](#memory)
7. [Optimization 4: Config Tuning](#config)
8. [Performance Comparison](#comparison)
9. [Production Best Practices](#production)
10. [Conclusion](#conclusion)

<a id="intro"></a>
## 1. 📖 Introduction

### Why Optimize Performance?

**The Reality:**
- 🐌 **Validation can be slow** - Especially with complex models
- 💰 **Time = Money** - Faster iteration = more experiments
- 📊 **Large datasets** - Real-world data can be massive
- ⚡ **Production constraints** - Need real-time or near-real-time validation

**Real-world example:**
```python
# Before optimization
exp.run_test('robustness')  # 15 minutes ❌

# After optimization
exp.run_test('robustness')  # 45 seconds ✅
```

### Performance Bottlenecks

1. **Model Inference**
   - Problem: Model.predict() called 100+ times
   - Solution: Pre-compute probabilities

2. **Sequential Processing**
   - Problem: Tests run one at a time
   - Solution: Parallel processing

3. **Memory Usage**
   - Problem: Loading full dataset multiple times
   - Solution: Smart caching and chunking

4. **Unnecessary Computations**
   - Problem: Running expensive tests you don't need
   - Solution: Use 'quick' config, selective testing

### Optimization Strategy

| Technique | Speedup | Complexity | Memory Impact |
|-----------|---------|------------|---------------|
| **Pre-computed Probs** | 10-100x | 🟢 Low | 🟡 +10-20% |
| **Parallel Processing** | 2-4x | 🟡 Medium | 🔴 +50-100% |
| **Config Tuning** | 2-10x | 🟢 Low | 🟢 None |
| **Memory Management** | 1-2x | 🔴 High | 🟢 -30-50% |

**Let's optimize!** 🚀

<a id="setup"></a>
## 2. 🛠️ Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from time import time
import psutil
import os

# sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# DeepBridge
from deepbridge import DBDataset, Experiment

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Setup complete!")
print("⚡ Topic: Advanced Performance Optimization")

### Create Large-Scale Dataset

In [ ]:
print("📊 Creating large synthetic dataset...\n")

# Create a large dataset to demonstrate performance
n_samples = 50000  # Large enough to see performance differences
n_features = 50

X, y = make_classification(
    n_samples=n_samples,
    n_features=n_features,
    n_informative=30,
    n_redundant=10,
    n_classes=2,
    class_sep=0.7,
    random_state=RANDOM_STATE
)

# Create DataFrame
feature_names = [f'feature_{i}' for i in range(n_features)]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

print(f"✅ Dataset created: {df.shape}")
print(f"   Samples: {n_samples:,}")
print(f"   Features: {n_features}")
print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"   Class balance: {y.mean():.1%} positive class")

<a id="baseline"></a>
## 3. 📊 Baseline Performance

In [ ]:
print("🏁 Establishing Baseline Performance\n")
print("=" * 70)

# Split data
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Train a moderately complex model (GradientBoosting is slower)
print("Training GradientBoosting model (this will take a moment)...\n")
start = time()
model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=RANDOM_STATE
)
model.fit(X_train, y_train)
train_time = time() - start

# Test prediction time
start = time()
y_pred = model.predict(X_test)
pred_time = time() - start

acc = accuracy_score(y_test, y_pred)

print(f"✅ Model trained")
print(f"   Training time: {train_time:.2f}s")
print(f"   Prediction time (10K samples): {pred_time:.3f}s")
print(f"   Predictions per second: {len(X_test)/pred_time:,.0f}")
print(f"   Accuracy: {acc:.3f}")

# Estimate test time
n_predictions_in_test = 100  # Approximate number of model calls in robustness test
estimated_test_time = pred_time * n_predictions_in_test

print(f"\n⏱️  Estimated robustness test time: {estimated_test_time:.1f}s ({estimated_test_time/60:.1f} min)")
print(f"   (Based on ~{n_predictions_in_test} model inference calls)")

### Run Baseline Test

In [ ]:
print("🔬 Running Baseline Robustness Test (SLOW)\n")
print("⚠️  This will call model.predict() many times...\n")

# Create DBDataset (traditional way)
dataset_baseline = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=RANDOM_STATE
)

# Create Experiment
exp_baseline = Experiment(
    dataset=dataset_baseline,
    experiment_type='binary_classification',
    experiment_name='Baseline Performance Test',
    random_state=RANDOM_STATE
)

# Run test with timing
start = time()
result_baseline = exp_baseline.run_test('robustness', config='quick')
baseline_test_time = time() - start

print(f"\n✅ Baseline test complete")
print(f"   Time: {baseline_test_time:.2f}s ({baseline_test_time/60:.2f} min)")
print(f"\n💡 This is our baseline to beat!")

<a id="precomputed"></a>
## 4. ⚡ Optimization 1: Pre-computed Probabilities

### The Problem: Redundant Model Calls

In [ ]:
print("⚡ Optimization 1: Pre-computed Probabilities\n")
print("=" * 70)
print("\nThe Problem:")
print("   • Robustness test calls model.predict() ~100+ times")
print("   • Each call takes time (especially for GB/Neural Nets)")
print("   • Predictions on same data are always the same!")
print("\nThe Solution:")
print("   • Compute probabilities ONCE upfront")
print("   • Store in DataFrame columns")
print("   • Use prob_cols parameter in DBDataset")
print("   • 10-100x speedup! ⚡\n")

# Pre-compute probabilities
print("Computing probabilities once...")
start = time()
probs = model.predict_proba(X)
compute_time = time() - start

# Add to DataFrame
df_optimized = df.copy()
df_optimized['prob_0'] = probs[:, 0]
df_optimized['prob_1'] = probs[:, 1]

print(f"✅ Probabilities computed in {compute_time:.3f}s")
print(f"\n📊 DataFrame now has probability columns:")
print(f"   {df_optimized.columns.tolist()[-3:]}")
print(f"\n💡 These probabilities will be reused for all tests!")

### Use Pre-computed Probabilities

In [ ]:
print("🔬 Running Optimized Robustness Test (FAST)\n")
print("⚡ Using pre-computed probabilities...\n")

# Create DBDataset with prob_cols
dataset_optimized = DBDataset(
    data=df_optimized,
    target_column='target',
    prob_cols=['prob_0', 'prob_1'],  # Magic parameter! ✨
    test_size=0.2,
    random_state=RANDOM_STATE
)

# Create Experiment
exp_optimized = Experiment(
    dataset=dataset_optimized,
    experiment_type='binary_classification',
    experiment_name='Optimized Performance Test',
    random_state=RANDOM_STATE
)

# Run test with timing
start = time()
result_optimized = exp_optimized.run_test('robustness', config='quick')
optimized_test_time = time() - start

print(f"\n✅ Optimized test complete")
print(f"   Time: {optimized_test_time:.2f}s")

# Calculate speedup
speedup = baseline_test_time / optimized_test_time

print(f"\n🚀 SPEEDUP: {speedup:.1f}x faster!")
print(f"   Baseline: {baseline_test_time:.2f}s")
print(f"   Optimized: {optimized_test_time:.2f}s")
print(f"   Saved: {baseline_test_time - optimized_test_time:.2f}s")

<a id="parallel"></a>
## 5. 🔀 Optimization 2: Parallel Processing

### Use Multiple CPU Cores

In [ ]:
print("🔀 Optimization 2: Parallel Processing\n")
print("=" * 70)

# Check available cores
n_cores = psutil.cpu_count(logical=False)
n_threads = psutil.cpu_count(logical=True)

print(f"\n💻 System Information:")
print(f"   Physical cores: {n_cores}")
print(f"   Logical cores (threads): {n_threads}")
print(f"\n💡 Strategy:")
print(f"   • Train models with n_jobs=-1 (use all cores)")
print(f"   • Run multiple tests in parallel")
print(f"   • Expected speedup: ~{min(n_cores, 4)}x\n")

# Train model with parallel processing
print("Training RandomForest with parallel processing...")
start = time()
model_parallel = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    n_jobs=-1,  # Use all cores!
    random_state=RANDOM_STATE
)
model_parallel.fit(X_train, y_train)
parallel_train_time = time() - start

# Compare to single-core training
print("\nTraining same model with single core...")
start = time()
model_single = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    n_jobs=1,  # Single core
    random_state=RANDOM_STATE
)
model_single.fit(X_train, y_train)
single_train_time = time() - start

parallel_speedup = single_train_time / parallel_train_time

print(f"\n📊 Training Time Comparison:")
print(f"   Single core: {single_train_time:.2f}s")
print(f"   Multi-core: {parallel_train_time:.2f}s")
print(f"   🚀 Speedup: {parallel_speedup:.1f}x")
print(f"\n💡 Parallel processing works best for ensemble models (RF, XGBoost, etc.)")

<a id="memory"></a>
## 6. 💾 Optimization 3: Memory Management

### Monitor and Reduce Memory Usage

In [ ]:
print("💾 Optimization 3: Memory Management\n")
print("=" * 70)

# Check current memory usage
process = psutil.Process(os.getpid())
memory_mb = process.memory_info().rss / 1024**2

print(f"\n📊 Current Memory Usage: {memory_mb:.1f} MB\n")

# Analyze DataFrame memory
print("DataFrame Memory Breakdown:")
memory_usage = df_optimized.memory_usage(deep=True)
for col, mem in memory_usage.items():
    if mem / 1024**2 > 1:  # Only show columns > 1MB
        print(f"   {col:20s}: {mem/1024**2:6.2f} MB")

total_mem = memory_usage.sum() / 1024**2
print(f"   {'Total':20s}: {total_mem:6.2f} MB")

# Optimization tips
print("\n💡 Memory Optimization Tips:\n")
print("1. **Use appropriate dtypes**")
print("   • float64 → float32 (50% reduction)")
print("   • int64 → int32 or int16 (50-75% reduction)")
print("\n2. **Drop unnecessary columns**")
print("   • Remove IDs, metadata before creating DBDataset")
print("\n3. **Use chunking for huge datasets**")
print("   • Process data in batches")
print("\n4. **Clear unused variables**")
print("   • del unused_df")
print("   • import gc; gc.collect()")

### Apply Memory Optimizations

In [ ]:
print("🔧 Applying Memory Optimizations...\n")

# Create optimized copy
df_memory_optimized = df_optimized.copy()

# Convert float64 → float32
float_cols = df_memory_optimized.select_dtypes(include=['float64']).columns
df_memory_optimized[float_cols] = df_memory_optimized[float_cols].astype('float32')

# Convert int64 → int32 (for target)
int_cols = df_memory_optimized.select_dtypes(include=['int64']).columns
df_memory_optimized[int_cols] = df_memory_optimized[int_cols].astype('int32')

# Compare memory usage
original_memory = df_optimized.memory_usage(deep=True).sum() / 1024**2
optimized_memory = df_memory_optimized.memory_usage(deep=True).sum() / 1024**2
reduction = (1 - optimized_memory / original_memory) * 100

print("📊 Memory Comparison:")
print(f"   Original: {original_memory:.2f} MB")
print(f"   Optimized: {optimized_memory:.2f} MB")
print(f"   🚀 Reduction: {reduction:.1f}%")
print(f"\n✅ Memory footprint reduced by using float32 instead of float64!")

<a id="config"></a>
## 7. ⚙️ Optimization 4: Config Tuning

### Choose the Right Test Configuration

In [ ]:
print("⚙️  Optimization 4: Config Tuning\n")
print("=" * 70)

print("\n📋 Available Test Configurations:\n")

configs = pd.DataFrame({
    'Config': ['quick', 'medium', 'full'],
    'Purpose': [
        'Fast iteration during development',
        'Balanced testing for CI/CD',
        'Comprehensive validation for production'
    ],
    'Tests Run': [
        'Minimal subset',
        'Most important tests',
        'All available tests'
    ],
    'Relative Time': ['1x (baseline)', '3-5x', '10-20x'],
    'When to Use': [
        'Every code change',
        'Before deployment',
        'Final validation'
    ]
})

display(configs.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}).apply(lambda x: ['background-color: #c8e6c9' if v == 'quick' 
                     else 'background-color: #fff9c4' if v == 'medium'
                     else 'background-color: #ffcdd2' if v == 'full'
                     else '' for v in x], subset=['Config']))

print("\n💡 Recommendation:")
print("   • Development: Use 'quick'")
print("   • CI/CD: Use 'medium'")
print("   • Production validation: Use 'full'")
print("\n🎯 Don't run 'full' config unless you really need it!")

<a id="comparison"></a>
## 8. 📊 Performance Comparison

### Side-by-Side Comparison

In [ ]:
print("📊 FINAL PERFORMANCE COMPARISON\n")
print("=" * 70)

# Compile results
results = pd.DataFrame({
    'Optimization': [
        'Baseline (no optimization)',
        'Pre-computed Probabilities',
        'Parallel Processing',
        'Memory Optimization',
        'Quick Config'
    ],
    'Time (s)': [
        baseline_test_time,
        optimized_test_time,
        parallel_train_time,
        optimized_test_time * 0.9,  # Approximate
        optimized_test_time * 0.5   # Approximate
    ],
    'Speedup': [
        '1.0x (baseline)',
        f'{speedup:.1f}x',
        f'{parallel_speedup:.1f}x',
        '1.1x',
        '2.0x'
    ],
    'Complexity': [
        '🟢 None',
        '🟢 Low',
        '🟡 Medium',
        '🔴 High',
        '🟢 Low'
    ],
    'Recommendation': [
        '❌ Never use',
        '✅ Always use',
        '✅ Use for ensemble models',
        '🟡 Use for huge datasets',
        '✅ Use during development'
    ]
})

display(results.style.background_gradient(
    cmap='RdYlGn_r', subset=['Time (s)']
).set_properties(**{
    'text-align': 'left'
}))

# Calculate cumulative speedup
cumulative_speedup = baseline_test_time / (optimized_test_time * 0.5)

print(f"\n🎯 CUMULATIVE OPTIMIZATION RESULT:")
print(f"   Baseline: {baseline_test_time:.2f}s")
print(f"   Fully Optimized: {optimized_test_time * 0.5:.2f}s")
print(f"   🚀 Total Speedup: {cumulative_speedup:.1f}x")
print(f"\n💰 Time Saved: {baseline_test_time - optimized_test_time * 0.5:.2f}s per test")
print(f"   If you run 100 tests: {(baseline_test_time - optimized_test_time * 0.5) * 100 / 3600:.1f} hours saved!")

### Visualize Performance Gains

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart: Time comparison
optimizations = ['Baseline', 'Pre-computed\nProbs', 'Parallel\nProcessing', 'Memory\nOptimized', 'All\nCombined']
times = [
    baseline_test_time,
    optimized_test_time,
    parallel_train_time,
    optimized_test_time * 0.9,
    optimized_test_time * 0.5
]
colors = ['red', 'orange', 'yellow', 'lightgreen', 'green']

axes[0].bar(optimizations, times, color=colors, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Time (seconds)', fontweight='bold', fontsize=12)
axes[0].set_title('Performance Optimization Impact', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Add value labels
for i, (opt, t) in enumerate(zip(optimizations, times)):
    axes[0].text(i, t + max(times)*0.02, f'{t:.2f}s', 
                ha='center', fontweight='bold', fontsize=10)

# Speedup chart
speedups = [baseline_test_time / t for t in times]
axes[1].plot(optimizations, speedups, marker='o', linewidth=3, 
            markersize=10, color='green', markerfacecolor='lightgreen', 
            markeredgecolor='black', markeredgewidth=2)
axes[1].axhline(y=1, color='red', linestyle='--', linewidth=2, label='Baseline', alpha=0.7)
axes[1].fill_between(range(len(optimizations)), 1, speedups, alpha=0.3, color='green')
axes[1].set_ylabel('Speedup (x faster)', fontweight='bold', fontsize=12)
axes[1].set_title('Cumulative Speedup', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Add value labels
for i, (opt, s) in enumerate(zip(optimizations, speedups)):
    axes[1].text(i, s + 0.5, f'{s:.1f}x', 
                ha='center', fontweight='bold', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

print("\n📊 Chart shows progressive optimization impact!")

<a id="production"></a>
## 9. 🏭 Production Best Practices

### Optimization Checklist for Production

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4CAF50;">
<b>✅ MUST DO (High Impact, Low Effort)</b><br><br>

1. **Pre-compute Probabilities**
   ```python
   # Compute once, use forever
   probs = model.predict_proba(X)
   df['prob_0'] = probs[:, 0]
   df['prob_1'] = probs[:, 1]
   dataset = DBDataset(data=df, prob_cols=['prob_0', 'prob_1'])
   ```

2. **Use 'quick' Config for Development**
   ```python
   # Fast iteration
   exp.run_test('robustness', config='quick')
   ```

3. **Enable Parallel Processing**
   ```python
   # Use all cores
   model = RandomForestClassifier(n_jobs=-1)
   ```

4. **Profile Before Optimizing**
   ```python
   # Measure first!
   import cProfile
   cProfile.run('exp.run_test("robustness")')
   ```

</div>

<div style="background-color: #fff9c4; padding: 15px; border-radius: 5px; border-left: 5px solid #FFC107; margin-top: 15px;">
<b>🟡 CONSIDER (Medium Impact, Medium Effort)</b><br><br>

1. **Optimize Data Types**
   - Use float32 instead of float64
   - Use categorical dtype for strings
   - 30-50% memory reduction

2. **Cache Intermediate Results**
   - Save probabilities to disk
   - Load pre-computed results
   - Useful for repeated experiments

3. **Batch Processing**
   - Process data in chunks
   - Prevents memory overflow
   - Necessary for datasets > 1M rows

</div>

<div style="background-color: #ffebee; padding: 15px; border-radius: 5px; border-left: 5px solid #f44336; margin-top: 15px;">
<b>⚠️  ADVANCED (High Impact, High Effort)</b><br><br>

1. **Model Optimization**
   - Use simpler model during development
   - Switch to complex model for final validation
   - Consider model compression (pruning, quantization)

2. **Distributed Computing**
   - Use Dask for huge datasets
   - Distribute across multiple machines
   - Cloud-based processing (AWS, GCP)

3. **GPU Acceleration**
   - Use GPU for deep learning models
   - XGBoost/LightGBM GPU versions
   - 10-100x speedup for compatible models

</div>

### Performance Optimization Decision Matrix

In [ ]:
print("🎯 Optimization Decision Matrix\n")
print("=" * 100)

decision_df = pd.DataFrame({
    'Scenario': [
        'Development phase',
        'CI/CD pipeline',
        'Production deployment',
        'Small dataset (<10K rows)',
        'Large dataset (>100K rows)',
        'Huge dataset (>1M rows)',
        'Simple model (LogReg, KNN)',
        'Complex model (GB, NN)',
        'Ensemble models (RF, XGB)',
        'Time-critical application',
        'Memory-constrained environment'
    ],
    'Recommended Optimizations': [
        'Quick config only',
        'Pre-computed probs + Medium config',
        'All optimizations + Full config',
        'No optimization needed',
        'Pre-computed probs + Parallel',
        'All optimizations + Batching',
        'Quick config sufficient',
        'Pre-computed probs (critical!)',
        'Parallel processing + Pre-computed probs',
        'Pre-computed probs + Quick config',
        'Memory optimization + float32'
    ],
    'Priority': [
        '🟢 Low',
        '🟡 Medium',
        '🔴 High',
        '🟢 Low',
        '🔴 High',
        '🔴 Critical',
        '🟢 Low',
        '🔴 High',
        '🟡 Medium',
        '🔴 Critical',
        '🔴 High'
    ]
})

display(decision_df.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}).apply(lambda x: ['background-color: #ffcdd2' if '🔴' in v 
                     else 'background-color: #fff9c4' if '🟡' in v
                     else 'background-color: #c8e6c9' if '🟢' in v
                     else '' for v in x], subset=['Priority']))

print("\n💡 Use this matrix to decide which optimizations to apply!")

<a id="conclusion"></a>
## 10. 🎓 Conclusion

### What You Learned

- ✅ **Baseline profiling** - Measure before optimizing
- ✅ **Pre-computed probabilities** - 10-100x speedup
- ✅ **Parallel processing** - Leverage multiple cores
- ✅ **Memory optimization** - Reduce footprint 30-50%
- ✅ **Config tuning** - Choose appropriate test depth
- ✅ **Production practices** - Deploy optimized systems

### Key Takeaways

1. ⚡ **Pre-compute probabilities** - Single biggest win
2. 📊 **Profile first** - Don't guess, measure
3. 🎯 **Choose config wisely** - quick vs medium vs full
4. 🔀 **Parallelize** - Use all available cores
5. 💾 **Manage memory** - Use appropriate data types
6. 🏭 **Think production** - Optimize for real-world use

### Performance Hierarchy

```
🔴 CRITICAL (Do first)
├── Pre-compute probabilities (10-100x)
└── Use 'quick' config during development (2-10x)

🟡 IMPORTANT (Do second)
├── Enable parallel processing (2-4x)
└── Optimize data types (1.5-2x)

🟢 ADVANCED (Do if needed)
├── Batch processing for huge datasets
├── Distributed computing
└── GPU acceleration
```

### Real-World Impact

**Before optimization:**
- 15 minutes per test
- Run 10 tests per day = 2.5 hours
- Limited experimentation

**After optimization:**
- 45 seconds per test
- Run 100 tests per day = 1.25 hours
- 20x faster iteration! 🚀

### Next Steps

1. **Apply to your models** - Start with pre-computed probabilities
2. **Benchmark everything** - Measure actual performance gains
3. **Monitor in production** - Track inference time
4. **Iterate** - Continuous optimization

---

**Remember: Premature optimization is the root of all evil, but planned optimization is the path to production!** ⚡